In [19]:
import sys
import time
from google import genai
from google.genai import types
from IPython.display import Markdown
from IPython.display import display
from dotenv import load_dotenv
import os
import glob
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry
from langcodes import Language
from collections import defaultdict
from tqdm import tqdm
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import re
from collections import defaultdict
from typing import List, Tuple
import fasttext
import re
from typing import Any, Dict, List, Tuple

In [20]:
def import_google_api():
    load_dotenv()
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

    client = genai.Client(api_key=GOOGLE_API_KEY)

    # Simple model check (keeping the original print logic)
    for m in client.models.list():
        if "embedContent" in m.supported_actions:
            print(m.name)

    return client

In [21]:
def embedding_function(client):
    class GeminiEmbeddingFunction(EmbeddingFunction):
        document_mode = True

        def __init__(self, client):
            self.client = client
            # Retry only on specific transient API errors
            self._retry = retry.Retry(predicate=lambda e: isinstance(e, genai.errors.APIError) and e.code in {429, 503})

        def __call__(self, input: Documents) -> Embeddings:
            embedding_task = "retrieval_document" if self.document_mode else "retrieval_query"
            response = self._retry(self.client.models.embed_content)(
                model="models/text-embedding-004",
                contents=input,
                config=types.EmbedContentConfig(task_type=embedding_task),
            )
            return [e.values for e in response.embeddings]

    return GeminiEmbeddingFunction(client)

In [22]:
# Assuming 'Document' is a class/dataclass with 'page_content' and 'metadata' attributes
# from a library like LangChain, LlamaIndex, etc.
class Document:
    def __init__(self, page_content: str, metadata: dict = None):
        self.page_content = page_content
        self.metadata = metadata if metadata is not None else {}

# NOTE: The original doc_stats, filename_base, google_drive_path, and all_chunks 
# are not defined in the provided snippet. I'll define placeholders or 
# make assumptions for a runnable example.

def parse_markdown_for_metadata(directory: str, google_drive_path: str = None) -> List[Document]:
    """
    Reads markdown files in a directory (and subdirectories) and creates a 
    single Document for each file, adding relevant metadata, but does not chunk.
    """
    markdown_files = glob.glob(os.path.join(directory, '**/*.md'), recursive=True)
    if not markdown_files:
        print("No markdown files found")
        return []

    print(f"Processing {len(markdown_files)} markdown files...")
    
    all_documents = []

    for filepath in tqdm(markdown_files, desc="Processing documents"):
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                markdown_text = f.read()
        except Exception as e:
            print(f"\nWarning: Could not read file {filepath}: {e}")
            continue

        # Extract basic file name for metadata
        filename_base = os.path.basename(filepath)

        # Create a single Document for the entire file content
        doc = Document(page_content=markdown_text)

        # Add metadata
        doc.metadata["source"] = filename_base
        # Use google_drive_path if provided, otherwise use local path
        doc.metadata["source_path"] = google_drive_path or filepath 
        
        # NOTE: Since we are not chunking by headers, we won't have a specific header.
        # We set it to an empty string or a placeholder.
        doc.metadata["header"] = "" 
        
        # Since the entire file is one document, these values reflect that.
        doc.metadata["chunk_index"] = 0
        doc.metadata["total_chunks"] = 1
        doc.metadata["is_complete_doc"] = True
        
        all_documents.append(doc)

    print(f"\nSuccessfully processed {len(all_documents)} files into documents.")
    return all_documents

In [23]:
def create_collection(chroma_client, gemini_embedding_function, documents_list):
    """
    Create or update ChromaDB collection with optimized batch processing.
    """
    DB_NAME = "hrstud-bot-en"
    embed_fn = gemini_embedding_function
    embed_fn.document_mode = True

    db = chroma_client.get_or_create_collection(
        name=DB_NAME,
        metadata={"model": "models/text-embedding-004", "dimension": 768},
        embedding_function=embed_fn
    )

    documents = [doc.page_content for doc in documents_list]
    metadatas = [doc.metadata for doc in documents_list]
    ids = [f"{DB_NAME}_doc_{i}" for i in range(len(documents))]

    if db.count() == 0:
        print(f"Adding {len(documents)} documents to ChromaDB collection: {DB_NAME}")

        # Optimized batch size for Gemini API
        BATCH_SIZE = 100
        
        for i in tqdm(range(0, len(documents), BATCH_SIZE), desc="Adding documents", unit="batch"):
            batch_end = min(i + BATCH_SIZE, len(documents))
            db.add(
                documents=documents[i:batch_end],
                metadatas=metadatas[i:batch_end],
                ids=ids[i:batch_end]
            )
            # Rate limiting for API stability
            time.sleep(0.2)

        print(f"\nCollection '{DB_NAME}' now contains {db.count()} documents.")
    else:
        print(f"Collection '{DB_NAME}' already has {db.count()} documents.")

In [24]:
def persistent_client(embed_fn):
    """
    Initialize persistent ChromaDB client.
    """
    persist_dir = "./output_en"
    chroma_client = chromadb.PersistentClient(path=persist_dir)

    DB_NAME = "hrstud-bot-en"
    collection = chroma_client.get_collection(DB_NAME, embedding_function=embed_fn)

    print(f"Connected to collection: {collection.name}")
    print(f"Documents: {collection.count()}")
    print(f"Metadata: {collection.metadata}")
    return embed_fn, collection

In [51]:
def get_article_en(user_query, embed_fn, collection, client):
    
    embed_fn.document_mode = False
    n_results_to_fetch = 7
    result = collection.query(query_texts=[user_query], n_results=n_results_to_fetch)
    
    all_passages = result["documents"][0]
    all_metadatas = result["metadatas"][0]

    query_oneline = user_query.replace("\n", " ")
    print(query_oneline)
    
    # Extract the main source link (from the first/most relevant result)
    main_source_link = all_metadatas[0].get("source_path", "Link not available")
    
    # Construct context
    context_list = []
    source_links = []  # Collect all unique source links
    
    for i, (passage, metadata) in enumerate(zip(all_passages, all_metadatas)):
        source_name = metadata.get("source", "Unknown source")
        source_path = metadata.get("source_path", "")
        
        # Collect unique sources for the bottom reference
        if source_path and source_path not in source_links:
            source_links.append(source_path)
        
        context_list.append(f"--- Source: {source_name} (Part {i+1} of {len(all_passages)}) ---\n{passage.strip()}")

    context = "\n\n".join(context_list)
    
    # Format sources for the bottom of the answer
    sources_text = "\n".join([f"- {link}" for link in source_links])
    
    prompt = f"""
    You are a kind, precise, and informative chatbot of the **Faculty of Croatian Studies**. 
    Your main task is to answer questions from students, prospective students, and staff about the faculty, 
    including information about study programs, courses, departments, admissions, and general school information.

    **CRITICAL RULES:**
    1.  Use ONLY the information provided in the supplied documentation.
    2.  Respond **in English**.
    3.  Be concise but complete — **synthesize all relevant details from ALL context sources into ONE cohesive answer**.
    4.  If the documentation does not contain the answer, clearly and politely state that you cannot find the answer 
        in the knowledge base and direct the user to contact the appropriate office.
    5.  Note if some classes are not offered in English.
    6.  **Do not use phrases like "Of course, I can help you!" or "Here is some information about...". 
        Start directly with the relevant answer.**
    7.  **IMPORTANT: Provide ONE unified answer, not multiple separate responses for each source.**

    **ANSWER FORMATTING:**
    * If there is ONE main source, start with: **Source: [Main Source Link]** followed by a blank line.
    * If there are MULTIPLE sources, provide the answer first, then at the end add a "**Sources:**" section listing all source links.
    * Use bold text for key terms (e.g., **Admissions**, **Philosophy**, **Head of Department**).
    * **When listing courses taught by a professor, organize them by level:**
    - **Undergraduate Courses:**
    - **Graduate Courses:**
    - **Doctoral Courses:**
    * For each course, include relevant details like ECTS credits, course hours, and language availability.
    * Responses should be professional and formal, yet polite in tone.
    * **Combine information from all sources into a single, coherent response.**
    * **DO NOT repeat "The source link is..." multiple times. Use it ONCE at the top if there's one main source, OR list all sources at the bottom.**

    **AVAILABLE DOCUMENTATION (Context):**
    {context}

    **USER QUESTION:** {query_oneline}

    **INSTRUCTIONS FOR SOURCE CITATION:**
    Main source link: {main_source_link}
    All source links: {sources_text}

    If the answer comes primarily from ONE source, start with "**Source:** {main_source_link}".
    If the answer uses MULTIPLE sources, end your response with:

    **Sources:**
    {sources_text}

    **ANSWER (provide ONE unified response with courses organized by academic level):**
    """
    
    answer = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "max_output_tokens": 2048,
            "temperature": 0.2,
            "top_p": 0.9
        }
    )

    return answer.text.strip()

In [26]:
# USAGE EXAMPLE - Uncomment to run

markdown_folder = "./markdown_en"
# 
# # STEP 1: Parse and chunk documents (run once or when documents change)
md_documents = parse_markdown_for_metadata(markdown_folder)
# 
# # STEP 2: Create collection and add documents (run once)
client = import_google_api()
gemini_embedding_function = embedding_function(client)
chroma_persistent_client = chromadb.PersistentClient(path="./output_en")
create_collection(chroma_persistent_client, gemini_embedding_function, md_documents)

Processing 666 markdown files...


Processing documents: 100%|██████████| 666/666 [00:00<00:00, 3140.73it/s]



Successfully processed 666 files into documents.
models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp
models/gemini-embedding-001
Collection 'hrstud-bot-en' already has 666 documents.


In [52]:
# STEP 3: Query the system (run for each query)
# 
client = import_google_api()
gemini_embedding_function = embedding_function(client)
embed_fn, collection = persistent_client(gemini_embedding_function)
# 
user_query = "Who is Sandro Skansi?"  # Example query
response = get_article_en(
    user_query=user_query,
    embed_fn=embed_fn,
    collection=collection,
    client=client,
)
display(Markdown(response))

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp
models/gemini-embedding-001
Connected to collection: hrstud-bot-en
Documents: 666
Metadata: {'dimension': 768, 'model': 'models/text-embedding-004'}
Who is Sandro Skansi?


**Sandro Skansi**, PhD, is the **Head** of the **Council of the Research Institute** at the Faculty of Croatian Studies. He is also a lecturer in charge of several courses. Please note that the lecturer is not able to offer courses in English at this time, except for Logic 2 and Social Philosophy where foreign students can attend additional office hours in English and take exams in English.

**Undergraduate Courses:**
*   **Social Media and Informal Logic** (Code: 284939): 3.0 ECTS, 15 hours of lectures and 15 hours of seminars. Offered in the 3rd and 5th semesters as an elective course for Philosophy and Culture students (FIL (1826): Izborni kolegiji).
*   **Contemporary Philosophy** (Code: 214821): 4.0 ECTS, 30 hours of lectures and 30 hours of seminars. Offered in the 1st semester as a mandatory course for Philosophy students.
*   **Introduction to Artificial Intelligence** (Code: 227468): 3.0 ECTS, 15 hours of lectures and 15 hours of seminars. Offered as an elective course for Psychology (PSI (2980) - izborni TZP) in the 2nd semester, for Croatology (KRO (3001) - izborni TZP) in the 2nd and 4th semesters, for Philosophy and Culture (FIL (1826): Izborni kolegiji) in the 4th and 6th semesters, and for Sociology (SOC (2960) - izborni TZP) in the 4th semester.
*   **Logic 2** (Code: 214825): 4.0 ECTS, 30 hours of lectures and 30 hours of seminars. Offered in the 3rd semester as a mandatory course for Philosophy and Culture students. Prerequisite: Passed Logic 1.
*   **Social Philosophy** (Code: 214837): 3.0 ECTS, 30 hours of lectures. Offered in the 5th semester as a mandatory course for Philosophy and Culture students.

**Graduate Courses:**
*   **Artificial Intelligence and Natural Language Understanding** (Code: 269284): 5.0 ECTS, 15 hours of lectures and 15 hours of seminars. Offered as an elective course for Philosophy students (FIL-dipl (17272) - Izborni kolegiji) in the 2nd and 4th semesters.

**Sources:**
*   https://www.fhs.hr/en/research_institute/contacts
*   https://www.fhs.hr/en/course/smail
*   https://www.fhs.hr/en/course/conphi_a
*   https://www.fhs.hr/en/course/itai
*   https://www.fhs.hr/en/course/aianlu
*   https://www.fhs.hr/en/course/log2_a
*   https://www.fhs.hr/en/course/socphi_b

In [ ]:
# ADVANCED: Test multiple queries
# 
test_queries = [
    "What classes does Marko Jerković teach",
    "What classes does Sandro Skansi teach?",
    "Who teaches Medieval European History?",
    "What History Undergraduate classes are offered in English?",
    "What is the number of Snježana Konovski?" # ili nema ili je premalo podataka na en stranici

]
# 
for query in test_queries:
    print(f"\n{'#'*60}")
    print(f"QUERY: {query}")
    print(f"{'#'*60}")
    response = get_article_en(
        user_query=query,
        embed_fn=embed_fn,
        collection=collection,
        client=client,
    )
    display(Markdown(response))
    print("\n")


############################################################
QUERY: What classes does Marko Jerković teach
############################################################
What classes does Marko Jerković teach


**Marko Jerković** teaches the following courses:

*   **Values of the Croatian constitutional order** (188090): This course is not offered in English. It is an elective course for students in Croatology (KRO) and History (POV). It is available in the 2nd, 4th, and 6th semesters. Course load includes 15 hours of lectures and 15 hours of seminars.
*   **Overview of Medieval Latinity** (200544): While all teaching activities are held in Croatian, foreign students in mixed groups have the opportunity to attend additional office hours with the lecturer and teaching assistants in English. The lecturer will also refer foreign students to corresponding literature in English and allow them to take exams in English. This is a mandatory course for Regular studij - Croatian Latinity in the 3rd semester. Course load includes 30 hours of lectures.
*   **Roman History and Civilization** (45755): In agreement with students, the lecturer will provide as many teaching elements in English as possible, or in both English and Croatian for mixed groups, including bilingual teaching materials and exams. Additional individual consultations are available for foreign students. It is a mandatory course for Regular studij - Latin language in the 1st semester and an elective course for students in History (POV) and Psychology (PSI) in the 1st semester. It is also an elective course for students in Croatology (KRO), History (POV), and Psychology (PSI) in the 3rd semester. Course load includes 15 hours of lectures and 15 hours of seminars.
*   **Croatian History (7-16 centuries)** (38053): While all teaching activities are held in Croatian, foreign students in mixed groups have the opportunity to attend additional office hours with the lecturer and teaching assistants in English. The lecturer will also refer foreign students to corresponding literature in English and allow them to take exams in English. This is a mandatory course for History students in the 2nd semester. Course load includes 30 hours of lectures.
*   **Medieval European History** (38043): While all teaching activities are held in Croatian, foreign students in mixed groups have the opportunity to attend additional office hours with the lecturer and teaching assistants in English. The lecturer will also refer foreign students to corresponding literature in English and allow them to take exams in English. This is a mandatory course for History students in the 2nd semester. Course load includes 30 hours of lectures.
*   **Cultural History and Latinity: The Middle and Early Modern Ages** (200546): This course is not offered in English. It is a mandatory course for Regular studij - Croatian Latinity in the 3rd semester. Course load includes 30 hours of lectures.
*   **Croatian Cultural and Political History of the Middle Ages** (37431): While all teaching activities are held in Croatian, foreign students in mixed groups have the opportunity to attend additional office hours with the lecturer and teaching assistants in English. The lecturer will also refer foreign students to corresponding literature in English and allow them to take exams in English. This is a mandatory course for Croatology students in the 1st semester. Course load includes 30 hours of lectures and 15 hours of seminars.

Consultations with **Marko Jerković** are held on Mondays from 13:00 to 14:00 in Location 103.

**Sources:**
*   [https://www.fhs.hr/en/course/votcco](https://www.fhs.hr/en/course/votcco)
*   [https://www.fhs.hr/en/course/ooml_a](https://www.fhs.hr/en/course/ooml_a)
*   [https://www.fhs.hr/en/course/rhac](https://www.fhs.hr/en/course/rhac)
*   [https://www.fhs.hr/en/course/ch7c](https://www.fhs.hr/en/course/ch7c)
*   [https://www.fhs.hr/en/course/meh](https://www.fhs.hr/en/course/meh)
*   [https://www.fhs.hr/en/course/chaltmaema](https://www.fhs.hr/en/course/chaltmaema)
*   [https://www.fhs.hr/en/course/ccaphotma](https://www.fhs.hr/en/course/ccaphotma)




############################################################
QUERY: What classes does Sandro Skansi teach?
############################################################
What classes does Sandro Skansi teach?


**Sandro Skansi** teaches the following courses:

*   **Artificial Intelligence and Natural Language Understanding** (AIANLU). This course is an elective for Philosophy students in the 2nd and 4th semesters of the Regular study program.
*   **Contemporary Philosophy**. This is a mandatory course for Philosophy students in their 1st semester.
*   **Introduction to Artificial Intelligence**. This course is an elective for students of Psychology (2nd semester), Croatology (2nd and 4th semesters), Philosophy and Culture (4th and 6th semesters), and Sociology (4th semester).
*   **Logic 2**. This is a mandatory course for Philosophy and Culture students in their 3rd semester and requires completion of Logic 1.
*   **Political Philosophy**. This course is a mandatory course for Philosophy students in their 1st semester and an elective for Communication Studies students in their 1st and 3rd semesters.
*   **Social Media and Informal Logic**. This course is an elective for Philosophy and Culture students in their 3rd and 5th semesters.
*   **Social Philosophy**. This is a mandatory course for Philosophy and Culture students in their 5th semester.

Please note that **Social Philosophy**, **Logic 2**, and **Political Philosophy** are taught in Croatian. However, foreign students have the opportunity to attend additional office hours in English and may be provided with literature and exams in English. The other courses are not offered in English at this time.

**Sources:**
*   [https://www.fhs.hr/en/course/conphi_a](https://www.fhs.hr/en/course/conphi_a)
*   [https://www.fhs.hr/en/course/smail](https://www.fhs.hr/en/course/smail)
*   [https://www.fhs.hr/en/course/aianlu](https://www.fhs.hr/en/course/aianlu)
*   [https://www.fhs.hr/en/course/socphi_b](https://www.fhs.hr/en/course/socphi_b)
*   [https://www.fhs.hr/en/course/itai](https://www.fhs.hr/en/course/itai)
*   [https://www.fhs.hr/en/course/log2_a](https://www.fhs.hr/en/course/log2_a)
*   [https://www.fhs.hr/en/course/polphi](https://www.fhs.hr/en/course/polphi)




############################################################
QUERY: Who teaches Medieval European History?
############################################################
Who teaches Medieval European History?


**Source:** https://www.fhs.hr/en/course/meh

The lecturer in charge of the **Medieval European History** course is izv. prof. dr. sc. **Marko Jerković**. The course is a mandatory course for regular **History** students in the 2nd semester. All teaching activities are held in Croatian. However, foreign students in mixed groups have the opportunity to attend additional office hours with the lecturer and teaching assistants in English to help master the course materials. Additionally, the lecturer will refer foreign students to the corresponding literature in English, as well as give them the possibility of taking the associated exams in English. His consultation schedule is Monday, 13.00-14.00 in location 103.




############################################################
QUERY: What History Undergraduate classes are offered in English?
############################################################
What History Undergraduate classes are offered in English?


The **Department of History** offers an undergraduate study program in **History** (single and double major) that aims to analyze the causal relationships between historical processes. 
While all teaching activities are generally held in Croatian, foreign students in mixed groups have the opportunity to attend additional office hours with the lecturer and teaching assistants in English to help master the course materials. 
The lecturer will also provide foreign students with corresponding literature in English and the possibility of taking exams in English.

Here are some courses offered in the undergraduate **History** program:

**1st Semester, 1st Year (Undergraduate):**
*   **History and Culture of Ancient Egypt (94019)**
*   **History and Culture of the Greek and Roman World (93908)**
*   **Methodology of Historical Science and Academic Literacy (130892)**
*   **Prehistory and the First Civilizations (93907)**

**2nd Semester, 1st Year (Undergraduate):**
*   **19th and 20th Century Croatian Historiography (38055)**
*   **Croatian History (7-16 centuries) (38053)**
*   **Medieval European History (38043)**

**3rd Semester, 2nd Year (Undergraduate):**
*   **Humanities in the Information Age (284996)**

**4th Semester, 2nd Year (Undergraduate):**
*   **19th Century History (38058)**
*   **Croatian History in the 19th Century (38059)**
*   **History of the USA from the First Colonies to the End of the 19th Century (284988)**

**5th Semester, 3rd Year (Undergraduate):**
*   **20th Century History (38065)**
*   **Resistance in Croatian history from the Early modern period to modern times (270149)**
*   **The Croatian People in the First Half of the 20th Century (188122)**

**6th Semester, 3rd Year (Undergraduate):**
*   **BA work (38774)**
*   **Croatia in the Second Half of the 20th Century (38067)**
*   **Croatian Heraldry and Semiotics of Coats of Arms (200603)**
*   **History of Austria (239951)**

Please note that the lecturer is currently unable to offer courses in English.

**Sources:**
*   [https://www.fhs.hr/en/undergraduate_study/history](https://www.fhs.hr/en/undergraduate_study/history)
*   [https://www.fhs.hr/en/undergraduate_study/double_major/history](https://www.fhs.hr/en/undergraduate_study/double_major/history)
*   [https://www.fhs.hr/en/course/emeawh](https://www.fhs.hr/en/course/emeawh)
*   [https://www.fhs.hr/en/course/1ch](https://www.fhs.hr/en/course/1ch)
*   [https://www.fhs.hr/en/course/cph](https://www.fhs.hr/en/course/cph)
*   [https://www.fhs.hr/en/course/hotuftfctteot1c_b](https://www.fhs.hr/en/course/hotuftfctteot1c_b)
*   [https://www.fhs.hr/en/course/rgioems](https://www.fhs.hr/en/course/rgioems)




############################################################
QUERY: What is the number of Snježana Konovski?
############################################################
What is the number of Snježana Konovski?


I am sorry, but I cannot find the phone number for Snježana Konovski in the provided documentation.

You may wish to contact the appropriate office for assistance.

**Sources:**
- fhs.hr_en_deanery_procurement_department.md
- fhs.hr_en_deanery_maintenance_service.md
- fhs.hr_en_deanery_legal_and_human_resources_service.md
- fhs.hr_en_course_bawor.md
- fhs.hr_en_course_fie1.md
- fhs.hr_en_course_smail.md
- fhs.hr_en_deanery_accounting_and_finance_service.md

In [39]:
# ADVANCED: Test multiple queries
# 
test_queries = [
    "What History Undergraduate classes are offered in English?",
    "What is the number of Snježana Konovski?"

]
# 
for query in test_queries:
    print(f"\n{'#'*60}")
    print(f"QUERY: {query}")
    print(f"{'#'*60}")
    response = get_article_en(
        user_query=query,
        embed_fn=embed_fn,
        collection=collection,
        client=client,
    )
    display(Markdown(response))
    print("\n")


############################################################
QUERY: What History Undergraduate classes are offered in English?
############################################################
What History Undergraduate classes are offered in English?


The Faculty of Croatian Studies offers undergraduate study programs in **History** as both a single major and a double major. The purpose of these programs is to analyze the causal relationships between historical processes and develop critical thinking skills.

Here are some courses offered in the **History (Single Major)** program for the academic year 2025/2026, with courses in English:

*   **1st Semester, 1st Year:**
    *   History and Culture of Ancient Egypt (94019)
    *   History and Culture of the Greek and Roman World (93908)
    *   Methodology of Historical Science and Academic Literacy (130892)
    *   Prehistory and the First Civilizations (93907)
    *   Ancient city on the Eastern Coast of the Adriatic (270102)
    *   Funeral Traditions and Architecture of Egyptian Civilization (158244)
    *   The Ancient Cities of Continental Croatia (270103)
    *   The Great Figures of Christianity: Between History, Tradition, and Myth (270101)
    *   Everyday Life in Ancient Rome (33169)
    *   Greco-Roman Religion (37562)
    *   Greek Influence on Roman Culture (86146)
    *   Roman Comedy (38874)
    *   Roman History and Civilization (45755)
*   **2nd Semester, 1st Year:**
    *   Croatian History (7-16 centuries) (38053)
    *   Medieval European History (38043)
    *   Contacts between civilizations in late antiquity and the early Middle Ages (209791)
    *   Croatian National Rulers (248673)
    *   Society and Religion: 13th - 15th Century (130475)
*   **3rd Semester, 2nd Year:**
    *   Croatian History (16th-18th Centuries) (38056)
    *   Croatian Latin Historiography (38057)
    *   Early Modern European and World History (38054)
    *   Humanities in the Information Age (284996)
    *   Catholic religious orders in Croatian culture (173127)
    *   Croatian Modern age in the eyes of Contemporaries (270105)
    *   Culture of Memory and Places of Remembrance (227641)
    *   History of Dubrovnik and the Dubrovnik Republic (214655)
    *   History of Modern Cities (284985)
    *   History of Russia (214657)
    *   History of the USA from the First Colonies to the End of the 19th Century (284988)
    *   The Profession of Historian (284998)
    *   Vlachs in European and Croatian culture (284989)
*   **4th Semester, 2nd Year:**
    *   19th Century History (38058)
    *   Croatian History in the 19th Century (38059)
    *   Introduction to the Auxiliary Sciences of History (153264)
    *   Catholic religious orders in Croatian culture (173127)
    *   Croatian Modern age in the eyes of Contemporaries (270105)
    *   Culture of Memory and Places of Remembrance (227641)
    *   History of Dubrovnik and the Dubrovnik Republic (214655)
    *   History of Modern Cities (284985)
    *   History of Russia (214657)
    *   History of the USA from the First Colonies to the End of the 19th Century (284988)
    *   The Profession of Historian (284998)
    *   Vlachs in European and Croatian culture (284989)
*   **5th Semester, 3rd Year:**
    *   20th Century History (38065)
    *   Resistance in Croatian history from the Early modern period to modern times (270149)
    *   The Croatian People in the First Half of the 20th Century (188122)
    *   Crimes of the Yugoslav Communist Regimes (213991)
    *   Croatian Hagiography (228732)
    *   Croatian History - Selected Topics 1929-1941 (202951)
    *   History and function of the International Criminal Tribunal for the former Yugoslavia (ICTY) in Hague (270197)
    *   Independent State of Croatia (214680)
    *   Military and police operations in the Croatian War of Independence 1991-1995 (270106)
    *   Military and Political Aspects of the Battle of Vukovar (270365)
    *   Military history of the second half of the 20th century (270196)
    *   State Leaders in the 20th century (270186)
    *   The history of Communism from Karl Marx to Fidel Castro (173129)
    *   World War II in feature film (285826)
*   **6th Semester, 3rd Year:**
    *   Croatia in the Second Half of the 20th Century (38067)
    *   Croatian Heraldry and Semiotics of Coats of Arms (200603)
    *   History of Austria (239951)
    *   Crimes of the Yugoslav Communist Regimes (213991)
    *   Croatian Hagiography (228732)
    *   Croatian History - Selected Topics 1929-1941 (202951)
    *   History and function of the International Criminal Tribunal for the former Yugoslavia (ICTY) in Hague (270197)
    *   Independent State of Croatia (214680)
    *   Military and police operations in the Croatian War of Independence 1991-1995 (270106)
    *   Military and Political Aspects of the Battle of Vukovar (270365)
    *   Military history of the second half of the 20th century (270196)
    *   State Leaders in the 20th century (270186)
    *   The history of Communism from Karl Marx to Fidel Castro (173129)
    *   World War II in feature film (285826)

While all teaching activities are held in Croatian, foreign students in mixed groups have the opportunity to attend additional office hours with the lecturer and teaching assistants in English to help master the course materials. The lecturer will also provide corresponding literature in English and allow exams to be taken in English. However, some courses like **History of the USA from the First Colonies to the End of the 19th Century** are not offered in English at this time.

**Sources:**
- fhs.hr_en_undergraduate_study_history.md
- fhs.hr_en_undergraduate_study_double_major_history.md
- fhs.hr_en_course_emeawh.md
- fhs.hr_en_course_1ch.md
- fhs.hr_en_course_cph.md
- fhs.hr_en_course_hotuftfctteot1c_b.md
- fhs.hr_en_course_rgioems.md




############################################################
QUERY: What is the number of Snježana Konovski?
############################################################
What is the number of Snježana Konovski?


I am sorry, but I cannot find information about Snježana Konovski in the knowledge base.

**Sources:**
- fhs.hr_en_deanery_procurement_department.md
- fhs.hr_en_deanery_maintenance_service.md
- fhs.hr_en_deanery_legal_and_human_resources_service.md
- fhs.hr_en_course_bawor.md
- fhs.hr_en_course_fie1.md
- fhs.hr_en_course_smail.md
- fhs.hr_en_deanery_accounting_and_finance_service.md